In [2]:
import time
from selenium import webdriver
from pprint import pprint
from bs4 import BeautifulSoup
import re

driver = webdriver.Chrome('chromedriver.exe')  # Optional argument, if not specified will search path.
driver.get('https://onefootball.com/pt-br/match/2269972');

C:\Users\user\AppData\Local\Temp\ipykernel_3248\336684140.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')  # Optional argument, if not specified will search path.


In [3]:
def get_events(list_of_elements):
    events_list = []
    for event in list_of_elements:
        event_text = event.text.split('\n')
        event_time = event_text[0]
        player_1 = event_text[1]
        if len(event_text) >= 3:
            player_2 = event_text[2]
        else:
            player_2 = ''
        event_detail = {
            'time': int(re.findall(r'\d+', event_time)[0]),
            'player_1': player_1,
            'player_2': player_2
        }
        events_list.append(event_detail)
    return events_list

def get_stats(list_of_elements):
    stats_list = []
    for stat in list_of_elements:
        stats_text = stat.text.split('\n')
        home_team_stat = stats_text[0]
        stat_name = stats_text[1]
        away_team_stat = stats_text[2]
        stats_detail = {
            'home_team_stat': home_team_stat,
            'away_team_stat': away_team_stat,
            'stat_name': stat_name
        }
        stats_list.append(stats_detail)
    return stats_list

## Aceitar os cookies

In [4]:
search_box = driver.find_elements_by_xpath("//button[contains(@class, 'didomi-components-button')]")
search_box[0].click()

C:\Users\user\AppData\Local\Temp\ipykernel_3248\2630487612.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//button[contains(@class, 'didomi-components-button')]")


In [5]:
search_box = driver.find_elements_by_xpath("//button[contains(@aria-label, 'Agree to our data processing and close')]")
search_box[0].click()

C:\Users\user\AppData\Local\Temp\ipykernel_3248\4220912207.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//button[contains(@aria-label, 'Agree to our data processing and close')]")


## Quem está jogando

In [6]:
search = driver.find_elements_by_xpath("//of-match-score-team")

C:\Users\user\AppData\Local\Temp\ipykernel_3248\4027852203.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//of-match-score-team")


In [7]:
search[0].text

'Barcelona Feminino'

In [8]:
def get_teams(list_of_elements):
    teams = {
        'home_team': list_of_elements[0].text,
        'away_team': list_of_elements[1].text,
    }
    return teams

In [9]:
teams = get_teams(search)

In [10]:
pprint(teams)

{'away_team': 'Real Madrid Feminino', 'home_team': 'Barcelona Feminino'}


## Placar

In [11]:
search = driver.find_elements_by_xpath("//p[contains(@class, 'match-score-scores')]")

C:\Users\user\AppData\Local\Temp\ipykernel_3248\335478207.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//p[contains(@class, 'match-score-scores')]")


In [13]:
search[0].text

'5\n:\n2'

In [14]:
def get_score(list_of_elements):
    score_element = list_of_elements[0].text.split('\n')
    scores = {
        'home_team_score': score_element[0],
        'away_team_score': score_element[2]
    }
    return scores

In [15]:
score = get_score(search)

In [16]:
pprint(score)

{'away_team_score': '2', 'home_team_score': '5'}


## Escalações

In [10]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [11]:
for tag in soup.findAll('span', attrs={'class': re.compile('match-lineup-v2__player-name')}):
    print(tag.text)

 6. Pina Medina 
 10. Hermoso 
 7. Graham Hansen 
 11. Putellas 
 12. Guijarro 
 14. Bonmatí 
 16. Rolfo 
 4. León 
 2. Paredes 
 8. Torrejón 
 1. Paños 


In [12]:
search_box = driver.find_elements_by_xpath("//button[contains(@title, '" + teams['away_team'] + "')]")

C:\Users\user\AppData\Local\Temp\ipykernel_16484\1664311966.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//button[contains(@title, '" + teams['away_team'] + "')]")


In [13]:
search_box

[<selenium.webdriver.remote.webelement.WebElement (session="2e3fe77028ff69dfe7b5447f9f18c4ea", element="41d5a9e1-52c1-4129-91ea-125dc2c2a79f")>]

In [14]:
search_box[0].click()

In [15]:
html = driver.page_source

In [16]:
html

'<html class="no-js" data-revision="23bb1cbbae21-prod" lang="pt-br" data-theme="auto"><head>\n  <meta charset="utf-8">\n  <title>Barcelona Feminino vs Real Madrid Feminino + “resultados”</title>\n  <base href="/">\n  <link href="https://api.onefootball.com" rel="preconnect">\n  <link href="https://image-service.onefootball.com" rel="preconnect">\n  <link href="https://oneftbl-cms.imgix.net" rel="preconnect">\n  <link href="https://pappagallu.onefootball.com" rel="preconnect">\n  <link href="https://web.localytics.com" rel="preconnect">\n  <link href="https://webanalytics.localytics.com" rel="preconnect">\n  <link href="/assets/images/favicon/manifest.json" rel="manifest">\n\n  <meta content="width=device-width, initial-scale=1" name="viewport">\n\n  <meta content="#fff" name="theme-color">\n  <meta content="light dark" name="color-scheme">\n  <link href="/assets/images/favicon/favicon.svg?20200629" rel="icon">\n  <link color="#1A1A24" href="/assets/images/favicon/favicon-masked.svg?202

In [17]:
soup = BeautifulSoup(html, "lxml")

In [18]:
for tag in soup.findAll('span', attrs={'class': re.compile('match-lineup-v2__player-name')}):
    print(tag.text)

 10. Gonzalez 
 7. Carmona García 
 8. Oroz Areta 
 22. del Castillo Belvide 
 21. Zornoza Sánchez 
 3. Abelleira Dueñas 
 23. Svava 
 5. Andrés 
 4. Peter 
 18. Rodríguez Herrero 
 1. Rodríguez Rivero 


## Eventos:

### Gol:
player_1 = quem fez o gol \
player_2 = assistência

### Cartão amarelo:
player_1 = quem recebeu o cartão amarelo \

### Penalti:
player_1 = quem fez o gol de penalti \
player_2 = "Penalti"

### Substituição:
player_1 = quem entra \
player_2 = quem sai

In [19]:
# Selecionar todos os eventos
search_box = driver.find_elements_by_xpath("//button[contains(@class, 'match-events__toggle-button')]")
# search_box
search_box[0].click()

C:\Users\user\AppData\Local\Temp\ipykernel_16484\3361420383.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//button[contains(@class, 'match-events__toggle-button')]")


In [20]:
# Listar todos os eventos da partida
search_box = driver.find_elements_by_xpath("//li[contains(@class, 'match-events__item')]")

C:\Users\user\AppData\Local\Temp\ipykernel_16484\1877595458.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search_box = driver.find_elements_by_xpath("//li[contains(@class, 'match-events__item')]")


In [21]:
events = get_events(search_box)

In [22]:
pprint(events)

[{'player_1': 'María León', 'player_2': 'Jennifer Hermoso', 'time': 8},
 {'player_1': 'Lucía Rodríguez', 'player_2': '', 'time': 12},
 {'player_1': 'Irene Paredes', 'player_2': '', 'time': 15},
 {'player_1': 'Olga Carmona', 'player_2': 'Pênalti', 'time': 16},
 {'player_1': 'Claudia Zornoza', 'player_2': 'Maite Oroz', 'time': 48},
 {'player_1': 'Aitana Bonmatí', 'player_2': 'Jennifer Hermoso', 'time': 52},
 {'player_1': 'Claudia Pina', 'player_2': 'Fridolina Rolfo', 'time': 55},
 {'player_1': 'Alexia Putellas',
  'player_2': 'Caroline Graham Hansen',
  'time': 62},
 {'player_1': 'Caroline Møller', 'player_2': 'Olga Carmona', 'time': 66},
 {'player_1': 'Nahikari García', 'player_2': 'Esther Gonzalez', 'time': 66},
 {'player_1': 'Caroline Graham Hansen', 'player_2': '', 'time': 70},
 {'player_1': 'Ingrid Syrstad Engen',
  'player_2': 'Patricia Guijarro',
  'time': 72},
 {'player_1': 'Leila Ouhabi', 'player_2': 'Jennifer Hermoso', 'time': 72},
 {'player_1': 'Ana-Maria Crnogorcevic',
  'pla

## Estatísticas

In [23]:
search = driver.find_elements_by_xpath("//li[contains(@class, 'match-stats__list-item')]")

C:\Users\user\AppData\Local\Temp\ipykernel_16484\2243698784.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//li[contains(@class, 'match-stats__list-item')]")


In [24]:
stats = get_stats(search)

In [25]:
pprint(stats)

[{'away_team_stat': '33%',
  'home_team_stat': '67%',
  'stat_name': 'POSSE DE BOLA'},
 {'away_team_stat': '5',
  'home_team_stat': '25',
  'stat_name': 'TOTAL DE CHUTES'},
 {'away_team_stat': '2', 'home_team_stat': '6', 'stat_name': 'CHUTES AO GOL'},
 {'away_team_stat': '40%',
  'home_team_stat': '60%',
  'stat_name': 'DISPUTAS GANHAS'}]


## Informações da partida

In [26]:
search = driver.find_elements_by_xpath("//li[contains(@class, 'match-info__entry')]")

C:\Users\user\AppData\Local\Temp\ipykernel_16484\1566266407.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  search = driver.find_elements_by_xpath("//li[contains(@class, 'match-info__entry')]")


In [27]:
def get_infos(list_of_elements):
    infos_list = []
    for info in list_of_elements:
        info_text = info.text.split('\n')
        info_title = info_text[0]
        info_content = info_text[1]
        info_detail = {
            'info_title': info_title,
            'info_content': info_content,
        }
        infos_list.append(info_detail)
    return infos_list

In [28]:
infos = get_infos(search)

In [29]:
pprint(infos)

[{'info_content': 'Liga dos Campeões Feminina da UEFA',
  'info_title': 'Competição'},
 {'info_content': '30/03/22', 'info_title': 'Início'},
 {'info_content': 'Camp Nou', 'info_title': 'Estádio'}]


In [30]:
of-match-score-team

NameError: name 'of' is not defined

In [ ]:
# driver.quit()